In [178]:
from os import walk
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
import cv2
import numpy as np
from math import log10, sqrt

%matplotlib inline

In [179]:
gray = 'IVUS Project/Grayscale Images'
aniso = 'IVUS Project/Anisotropic Filter Images'
path = 'IVUS Project/'
total_folders = 10

In [180]:
files_gray = []
for root,dirname,filenames in walk(gray):
    for file in filenames:
        req_path = '/'.join([root,file])
        files_gray.append(req_path)
files_gray.sort()
files_gray = files_gray * (total_folders - 1)
files_gray

['IVUS Project/Grayscale Images/anonymus 00001_Frame_1.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_13.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_14.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_15.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_18.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_19.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_2.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_4.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_8.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_9.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_1.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_13.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_14.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_15.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_18.png',
 'IVUS Project/Grayscale Images/anonymus 00001_Frame_19.png',
 'IVUS Project

In [181]:
files_aniso = []
for root,dirname,filenames in walk(aniso):
    for file in filenames:
        req_path = '/'.join([root,file])
        files_aniso.append(req_path)
files_aniso.sort()
files_aniso

['IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_1.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_13.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_14.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_15.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_18.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_19.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_2.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_4.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_8.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_9.png']

In [182]:
files = []
for root,dirname,filenames in walk(path):
    if root == 'IVUS Project/Grayscale Images':
        continue
    for file in filenames:
        req_path = '/'.join([root,file])
        files.append(req_path)
files.sort()
files

['IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_1.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_13.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_14.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_15.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_18.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_19.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_2.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_4.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_8.png',
 'IVUS Project/Anisotropic Filter Images/anonymus 00001_Frame_9.png',
 'IVUS Project/Bilateral Filter Images/anonymus 00001_Frame_1.png',
 'IVUS Project/Bilateral Filter Images/anonymus 00001_Frame_13.png',
 'IVUS Project/Bilateral Filter Images/anonymus 00001_Frame_14.png',
 'IVUS Project/Bilateral Filter Images/anonymus 00001_Frame_15.png',
 'IVUS Project/Bilat

In [191]:
df = pd.DataFrame(data=np.column_stack((files_gray,files)),columns=['gray','other'])

df["gray"] = df["gray"].astype(str)
df["other"] = df["other"].astype(str)

df

,gray,other
0,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...
1,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...
2,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...
3,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...
4,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...
...,...,...
85,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...
86,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...
87,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...
88,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...


In [192]:
def PSNR(gray, other):
    mse = np.mean((gray - other) ** 2)
    if(mse == 0):  
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr
  
def main_psnr(x,y):
    gray = cv2.imread(x)
    other = cv2.imread(y, 1)
    value = PSNR(gray, other)
    return value

In [193]:
df['PSNR'] = [main_psnr(str(df.gray[i]),str(df.other[i])) for i in range(df.shape[0])]
df

,gray,other,PSNR
0,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.311754
1,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.058068
2,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.026557
3,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,43.969402
4,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.191154
...,...,...,...
85,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.561255
86,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.758577
87,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.578374
88,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.583139


In [194]:
def mse(gray, other):
    mse = np.mean((gray - other) ** 2)
    return mse
  
def main_mse(x,y):
    gray = cv2.imread(x)
    other = cv2.imread(y, 1)
    value = mse(gray, other)
    return value

In [199]:
df['mse'] = [main_mse(str(df.gray[i]),str(df.other[i])) for i in range(df.shape[0])]
df

,gray,other,PSNR,mse
0,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.311754,2.409378
1,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.058068,2.554310
2,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.026557,2.572910
3,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,43.969402,2.606995
4,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.191154,2.477222
...,...,...,...,...
85,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.561255,36.054119
86,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.758577,34.452656
87,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.578374,35.912281
88,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.583139,35.872898


In [202]:
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    img1 = cv2.imread(img1)
    img2 = cv2.imread(img2)
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [203]:
df['ssim'] = [calculate_ssim(str(df.gray[i]),str(df.other[i])) for i in range(df.shape[0])]
df

,gray,other,PSNR,mse,ssim
0,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.311754,2.409378,0.993116
1,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.058068,2.554310,0.993066
2,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.026557,2.572910,0.993070
3,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,43.969402,2.606995,0.993006
4,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Anisotropic Filter Images/anonymu...,44.191154,2.477222,0.993425
...,...,...,...,...,...
85,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.561255,36.054119,0.205623
86,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.758577,34.452656,0.223520
87,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.578374,35.912281,0.228551
88,IVUS Project/Grayscale Images/anonymus 00001_F...,IVUS Project/Wiener Filter Images/anonymus 000...,32.583139,35.872898,0.226285


In [208]:
df.to_csv('data.csv')

In [209]:
df.to_excel('data.xlsx')